In [1]:
import pandas as pd
import numpy as np
import pickle
import random

In [2]:
with open('../models/optimized_XGB.pickle', 'rb') as f:
    xgb_clf = pickle.load(f)

In [3]:
df = pd.read_csv('../data/raw/test.csv.gz', compression='gzip')

C:\Users\marco\anaconda3\envs\capstone2env\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
df.head()

,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,...,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier
0,0000010489e3af074adeac69c53e555e,win8defender,1.1.15400.5,4.18.1810.5,1.281.501.0,0,7.0,0,NaN,53447.0,...,807.0,8554.0,1,NaN,0.0,0,0,0.0,0.0,7.0
1,00000176ac758d54827acd545b6315a5,win8defender,1.1.15400.4,4.18.1809.2,1.279.301.0,0,7.0,0,NaN,53447.0,...,554.0,33105.0,1,NaN,0.0,0,0,0.0,1.0,12.0
2,0000019dcefc128c2d4387c1273dae1d,win8defender,1.1.15300.6,4.18.1809.2,1.277.230.0,0,7.0,0,NaN,49480.0,...,556.0,63396.0,1,NaN,0.0,0,0,0.0,1.0,11.0
3,0000055553dc51b1295785415f1a224d,win8defender,1.1.15400.5,4.18.1810.5,1.281.664.0,0,7.0,0,NaN,42160.0,...,628.0,26320.0,1,0.0,0.0,0,0,0.0,0.0,10.0
4,00000574cefffeca83ec8adf9285b2bf,win8defender,1.1.15400.4,4.18.1809.2,1.279.236.0,0,7.0,0,NaN,53447.0,...,556.0,63269.0,1,NaN,0.0,0,0,0.0,1.0,3.0


In [5]:
df.shape

(7853253, 82)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7853253 entries, 0 to 7853252
Data columns (total 82 columns):
 #   Column                                             Dtype  
---  ------                                             -----  
 0   MachineIdentifier                                  object 
 1   ProductName                                        object 
 2   EngineVersion                                      object 
 3   AppVersion                                         object 
 4   AvSigVersion                                       object 
 5   IsBeta                                             int64  
 6   RtpStateBitfield                                   float64
 7   IsSxsPassiveMode                                   int64  
 8   DefaultBrowsersIdentifier                          float64
 9   AVProductStatesIdentifier                          float64
 10  AVProductsInstalled                                float64
 11  AVProductsEnabled                                 

In [7]:
ID_sub = df.MachineIdentifier
df.drop('MachineIdentifier', inplace=True, axis=1)

In [8]:
to_drop = ['DefaultBrowsersIdentifier',
           'PuaMode',
           'Census_ProcessorClass',
           'Census_InternalBatteryType',
           'Census_IsFlightingInternal',
           'Census_ThresholdOptIn',
           'Census_IsWIMBootEnabled']

In [9]:
# Categorical Features Processing
df.loc[df.ProductName == 'scep', 'ProductName'] = np.nan
df.loc[df.Processor == 'arm64', 'Processor'] = np.nan
df.loc[df.OsVer == '6.1.0.0', 'OsVer'] = '6.1.1.0'
df.loc[df.OsVer == '10.0.3.0', 'OsVer'] = '10.0.0.0'
df.loc[df.OsVer == '10.0.0.2', 'OsVer'] = '10.0.0.0'
df.loc[df.OsVer == '10.0.7.0', 'OsVer'] = '10.0.0.0'
df.loc[df.OsVer == '10.0.1.0', 'OsVer'] = '10.0.0.0'
df.loc[df.SmartScreen == 'on', 'SmartScreen'] = 'On'
df.loc[df.SmartScreen == 'off', 'SmartScreen'] = 'Off'
df.loc[df.SmartScreen == '&#x01;', 'SmartScreen'] = '&#x0_'
df.loc[df.SmartScreen == '&#x02;', 'SmartScreen'] = '&#x0_'
df.loc[df.Census_DeviceFamily == 'Windows', 'Census_DeviceFamily'] = np.nan
df.loc[df.Census_PrimaryDiskTypeName == 'UNKNOWN', 'Census_PrimaryDiskTypeName'] = 'Unspecified'
df.loc[df.Census_ChassisTypeName == 'UNKNOWN', 'Census_ChassisTypeName'] = 'Unknown'
df.loc[df.Census_OSArchitecture == 'arm64', 'Census_OSArchitecture'] = np.nan

In [10]:
# Numerical Features Processing
#cat_col = df.select_dtypes('object').columns.tolist()
#num_col = [col for col in df.columns.tolist() if col not in cat_col]
#binary_feat = [col for col in num_col if df[col].nunique() <= 2]
df.IsProtected = df.IsProtected.map({np.nan:0, 0:1, 1:2})

to_drop = to_drop + ['IsBeta',
                     'AutoSampleOptIn',
                     'SMode',
                     'Census_IsPortableOperatingSystem',
                     'Census_IsFlightsDisabled',
                     'Census_IsVirtualDevice']

limit = 16*10**6
df.loc[df['Census_PrimaryDiskTotalCapacity'] > limit, 'Census_PrimaryDiskTotalCapacity'] = np.nan

vol_limit = 2e6
df.loc[df['Census_SystemVolumeTotalCapacity'] > vol_limit, 'Census_SystemVolumeTotalCapacity'] = np.nan

df['Census_OSBranch_release_extracted'] = df.OsBuildLab.str.split('.').str.get(3)
to_drop.append('OsBuildLab')
to_drop.append('Census_OSBuildNumber')
to_drop.append('Census_OSBranch')
to_drop.append('UacLuaenable')

df.shape

(7853253, 82)

In [11]:
to_drop

['DefaultBrowsersIdentifier',
 'PuaMode',
 'Census_ProcessorClass',
 'Census_InternalBatteryType',
 'Census_IsFlightingInternal',
 'Census_ThresholdOptIn',
 'Census_IsWIMBootEnabled',
 'IsBeta',
 'AutoSampleOptIn',
 'SMode',
 'Census_IsPortableOperatingSystem',
 'Census_IsFlightsDisabled',
 'Census_IsVirtualDevice',
 'OsBuildLab',
 'Census_OSBuildNumber',
 'Census_OSBranch',
 'UacLuaenable']

In [12]:
df.drop(to_drop, axis=1, inplace=True)
df.shape

(7853253, 65)

In [13]:
numerical_col = ['Census_ProcessorCoreCount',
                 'Census_PrimaryDiskTotalCapacity',
                 'Census_SystemVolumeTotalCapacity',
                 'Census_TotalPhysicalRAM',
                 'Census_InternalPrimaryDiagonalDisplaySizeInInches',
                 'Census_InternalPrimaryDisplayResolutionHorizontal',
                 'Census_InternalPrimaryDisplayResolutionVertical',
                 'Census_InternalBatteryNumberOfCharges']
binary_col = [col for col in binary_feat if col not in to_drop]
categorical_col = [col for col in df.columns.tolist() if col not in to_drop
                  if col not in numerical_col if col not in binary_col]

tot_col = numerical_col + binary_col + categorical_col
len(tot_col)

65

In [14]:
high_cardinality = ['Census_OEMModelIdentifier', 'CityIdentifier',
                    'Census_FirmwareVersionIdentifier', 'AvSigVersion',
                    'AVProductStatesIdentifier', 'Census_ProcessorModelIdentifier',
                    'Census_OEMNameIdentifier', 'Census_OSVersion', 'GeoNameIdentifier',
                    'CountryIdentifier', 'Census_OSBuildRevision',
                    'LocaleEnglishNameIdentifier', 'Census_FirmwareManufacturerIdentifier',
                    'IeVerIdentifier', 'AppVersion', 'Census_OSUILocaleIdentifier',
                    'EngineVersion', 'Census_OSInstallLanguageIdentifier',
                    'OrganizationIdentifier', 'Census_OSBranch_release_extracted',
                    'OsBuild', 'Census_ChassisTypeName', 'Census_OSEdition',
                    'Census_OSSkuName']

In [15]:
#modified from: https://wrosinski.github.io/fe_categorical_encoding/
        
def labelcount_encode(X, categorical_features, ascending=False):
    print('LabelCount encoding: {}'.format(categorical_features))
    X_ = pd.DataFrame()
    for cat_feature in categorical_features:
        cat_feature_value_counts = X[cat_feature].fillna(0).value_counts()
        value_counts_list = cat_feature_value_counts.index.tolist()
        if ascending:
            # for ascending ordering
            value_counts_range = list(
                reversed(range(len(cat_feature_value_counts))))
        else:
            # for descending ordering
            value_counts_range = list(range(len(cat_feature_value_counts)))
        labelcount_dict = dict(zip(value_counts_list, value_counts_range))
        X_[cat_feature] = X[cat_feature].fillna(0).map(
            labelcount_dict)
    X_ = X_.add_suffix('_encoded')
    X_ = X_.astype(np.uint32)
    return X_

df_encoded = labelcount_encode(df, high_cardinality)
df_encoded.shape, df.shape

LabelCount encoding: ['Census_OEMModelIdentifier', 'CityIdentifier', 'Census_FirmwareVersionIdentifier', 'AvSigVersion', 'AVProductStatesIdentifier', 'Census_ProcessorModelIdentifier', 'Census_OEMNameIdentifier', 'Census_OSVersion', 'GeoNameIdentifier', 'CountryIdentifier', 'Census_OSBuildRevision', 'LocaleEnglishNameIdentifier', 'Census_FirmwareManufacturerIdentifier', 'IeVerIdentifier', 'AppVersion', 'Census_OSUILocaleIdentifier', 'EngineVersion', 'Census_OSInstallLanguageIdentifier', 'OrganizationIdentifier', 'Census_OSBranch_release_extracted', 'OsBuild', 'Census_ChassisTypeName', 'Census_OSEdition', 'Census_OSSkuName']


((7853253, 24), (7853253, 65))

In [16]:
df_clean = pd.concat([df.drop(high_cardinality, axis=1), df_encoded], axis=1)
df_clean.shape

(7853253, 65)

In [ ]:
with open('../data/processed/X_train', 'rb') as f:
    X_train = pickle.load(f)

In [22]:
X_test = df_clean[X_train.columns]
X_test.head()

,Census_ProcessorCoreCount,Census_PrimaryDiskTotalCapacity,Census_SystemVolumeTotalCapacity,Census_TotalPhysicalRAM,Census_InternalPrimaryDiagonalDisplaySizeInInches,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,Census_InternalBatteryNumberOfCharges,Census_OEMModelIdentifier_encoded,CityIdentifier_encoded,...,Census_GenuineStateName,Census_ActivationChannel,Census_FlightRing,RtpStateBitfield,AVProductsInstalled,AVProductsEnabled,OsSuite,IsProtected,Census_ProcessorManufacturerIdentifier,Wdft_RegionIdentifier
0,4.0,488386.0,123179.0,8192.0,15.5,1920.0,1080.0,8.0,43478,113,...,IS_GENUINE,OEM:DM,Retail,7.0,1.0,1.0,768,2,5.0,7.0
1,4.0,1907729.0,1882352.0,8192.0,15.5,1366.0,768.0,0.0,1,3923,...,IS_GENUINE,Retail,Retail,7.0,1.0,1.0,768,2,5.0,12.0
2,2.0,29820.0,28678.0,4096.0,13.9,1280.0,720.0,0.0,54,5,...,IS_GENUINE,OEM:DM,Retail,7.0,2.0,1.0,768,2,5.0,11.0
3,2.0,476940.0,439345.0,4096.0,14.0,1366.0,768.0,0.0,3192,62,...,IS_GENUINE,OEM:DM,Retail,7.0,2.0,1.0,768,2,5.0,10.0
4,4.0,476940.0,461506.0,2048.0,15.5,1366.0,768.0,0.0,73,28,...,IS_GENUINE,Retail,Retail,7.0,1.0,1.0,768,2,5.0,3.0


In [25]:
y_pred = xgb_clf.predict_proba(X_test)
y_pred

C:\Users\marco\anaconda3\envs\capstone2env\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


array([[0.38091034, 0.61908966],
       [0.2410866 , 0.7589134 ],
       [0.48372263, 0.5162774 ],
       ...,
       [0.5596155 , 0.44038448],
       [0.3182913 , 0.6817087 ],
       [0.48380065, 0.51619935]], dtype=float32)

In [36]:
# Save test predictions to file
output = pd.DataFrame({'MachineIdentifier': ID_sub,
                       'HasDetections': y_pred[:, 1]})
output.to_csv('../docs/xgb_sub.csv', index=False)

In [35]:
output.head()

,Id,HasDetections
0,0000010489e3af074adeac69c53e555e,0.619090
1,00000176ac758d54827acd545b6315a5,0.758913
2,0000019dcefc128c2d4387c1273dae1d,0.516277
3,0000055553dc51b1295785415f1a224d,0.410218
4,00000574cefffeca83ec8adf9285b2bf,0.700798
